# Modelos de prediccion
Vamos a probar distintos enfoques:
Enfoques de serie de tiempo:
* Suavizado exponencial #investigar

Enfoques Machine Learning:
* XGboost: Emiii
* LIGHT_GBM: Rugo
* Catboost: Vano

In [1]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test

from DataLoader import generar_rolling_features,crear_archivo_kaggle
import xgboost as xgb
import pandas as pd

In [2]:
ventas=importar_ventas()

In [5]:
ventas.columns

Index(['date', 'demand', 'mean_price', 'std_price', 'max_price', 'min_price',
       'std_discount', 'max_discount', 'min_discount', 'std_diff_factory',
       'max_diff_factory', 'min_diff_factory', 'group', 'category', 'cluster',
       'store_cod', 'subgroup_cod', 'day', 'month', 'year'],
      dtype='object')

In [3]:
test=preparar_test("ids_test.csv")
#test

In [4]:
agregar_fourier(ventas,2)
agregar_fourier(test,2)


In [5]:
ventas=generar_rolling_features(ventas,"mean_price",[1,2,3,4],["subgroup_cod","store_cod"])
ventas=generar_rolling_features(ventas,"demand",[1,2,3,4],["subgroup_cod","store_cod"])

In [7]:
#busquemos otra fecha para probar esto objetivo 01-01-2023 a 07-01-2023

# ventas_2023=ventas[ventas["date"]<"02-01-2023"]
# test_2023=ventas_2023.groupby(["subgroup_cod","store_cod"]).tail(7)
# train_2023=ventas_2023.drop(test_2023.index)

ventas.columns

Index(['date', 'demand', 'mean_price', 'std_price', 'max_price', 'min_price',
       'cluster', 'store_cod', 'subgroup_cod', 'demand_lag_1', 'demand_lag_2',
       'demand_lag_3', 'demand_lag_4', 'demand_lag_5', 'demand_lag_6',
       'demand_lag_7', 'day', 'month', 'year', 'anual_sin_1', 'anual_cos_1',
       'mensual_sin_1', 'mensual_cos_1', 'anual_sin_2', 'anual_cos_2',
       'mensual_sin_2', 'mensual_cos_2', 'mean_price_rolling_mean_7d',
       'mean_price_rolling_sum_7d', 'mean_price_rolling_mean_14d',
       'mean_price_rolling_sum_14d', 'mean_price_rolling_mean_21d',
       'mean_price_rolling_sum_21d', 'mean_price_rolling_mean_28d',
       'mean_price_rolling_sum_28d', 'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 'demand_rolling_mean_14d',
       'demand_rolling_sum_14d', 'demand_rolling_mean_21d',
       'demand_rolling_sum_21d', 'demand_rolling_mean_28d',
       'demand_rolling_sum_28d'],
      dtype='object')

In [6]:
import numpy as np
FEATURES=[
   #'mean_price', 
    'cluster', 
    #'store_cod', 
    'subgroup_cod',
    #'demand_lag_1', #'demand_lag_2', 'demand_lag_3', 
    #'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
   'mensual_sin_1', 
    'mensual_cos_1',
   #'anual_sin_2', 
    #'anual_cos_2',
    'mensual_sin_2', 
    #'mensual_cos_2',
   # "std_price",
    "max_price", 
   "min_price",
    #'mean_price_rolling_mean_7d',
   #    'mean_price_rolling_sum_7d',# 'mean_price_rolling_mean_14d',
       #'mean_price_rolling_sum_14d', 
   # 'mean_price_rolling_mean_21d',
     #  'mean_price_rolling_sum_21d', 'mean_price_rolling_mean_28d',
    #  'mean_price_rolling_sum_28d', 
    #'demand_rolling_mean_7d',
    #  'demand_rolling_sum_7d', 
    'demand_rolling_mean_14d',
     #  'demand_rolling_sum_14d', #'demand_rolling_mean_21d',
       #'demand_rolling_sum_21d', #'demand_rolling_mean_28d',
      # 'demand_rolling_sum_28d'
    'std_discount', 'max_discount', 'min_discount', 'std_diff_factory',
    'max_diff_factory', 'min_diff_factory'
] #recordar que ya probamos con fourier 3 y 4 y fue un fracaso

TARGET=['demand']

test_interno=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas
train_interno=ventas.drop(test_interno.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test_real=test[FEATURES]

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]

#y_train_interno_log=np.log1p(y_train_interno)
#y_test_interno_log=np.log1p(y_test_interno)


# X_test_2023=test_2023[FEATURES]
# y_test_2023=test_2023[TARGET]

# X_train_2023=train_2023[FEATURES]
# y_train_2023=train_2023[TARGET]

## Modelo interno

In [ ]:
from sklearn.metrics import r2_score

reg = xgb.XGBRegressor(
    n_estimators=1000,
    eval_metric="rmse",
    eta=0.05,
    max_depth=6,
   #colsample_bytree=0.9, #reducir el numero de columnas aumenta la importancia de std_price
    early_stopping_rounds=50 
)

reg.fit(
    X_train_interno, y_train_interno
   ,eval_set=[(X_test_interno, y_test_interno)]   
   ,verbose=50                       
)
#sin anual_sin_2 450 estimadores =0.11283 y con da lo mismo

[0]	validation_0-rmse:3.17350
[50]	validation_0-rmse:2.16045
[100]	validation_0-rmse:2.04869
[150]	validation_0-rmse:1.94115


In [40]:
pd.DataFrame(reg.feature_importances_,columns=["importance"],index=reg.feature_names_in_).sort_values(by="importance",ascending=False)

,importance
max_price,0.263704
min_price,0.209086
demand_rolling_mean_14d,0.083938
year,0.069071
mensual_cos_1,0.066970
demand_rolling_sum_7d,0.059225
demand_rolling_sum_14d,0.055347
anual_sin_1,0.041263
anual_cos_1,0.027554
cluster,0.027519


## Modelo kraggel


In [8]:
from sklearn.metrics import mean_squared_error
import numpy as np

prediccion=reg.predict(X_test_interno)


# y_true_log y y_pred_log están en escala log1p
y_true_original = np.expm1(y_test_interno_log)
y_pred_original = np.expm1(prediccion)

rmse_original = np.sqrt(mean_squared_error(y_true_original, y_pred_original))

print(rmse_original)
print(r2_score(y_true_original,y_pred_original))
orden_bueno = pd.DataFrame({
    "store_subgroup_date_id": test["store_subgroup_date_id"],
    "prediccion": prediccion.ravel()
})

kaggle=crear_archivo_kaggle(orden_bueno)

NameError: name 'reg' is not defined

In [11]:
#kaggle.describe()
400*48/60/60

5.333333333333333

In [48]:
ventas_computing=generar_rolling_features(ventas_computing,"demand",[1,4],["subgroup_cod","store_cod"])